### Task 4

<span style="color: green; font-size: 14px;">
На предыдущих этапах ты занимался просто подготовкой данных. Ты не получил никакой аналитической информации на основе имеющихся данных. Пришло время приступить к анализу. Помни, что, согласно нашему предположению, пользователи приступили бы к работе над лабораторными заданиями раньше, если бы они просмотрели ленту новостей? Это означает, что ключевой метрикой для нас является период времени (delta) между датой начала работы пользователя над лабораторным заданием (первого коммита) и сроком сдачи лабораторного задания. Мы будем смотреть на то, меняется ли она в зависимости от просмотра страницы.

Что тебе нужно сделать в этом упражнении:
</span>
<span style="color: green; font-size: 13px;">

1. Создай соединение с базой данных с помощью библиотеки `sqlite3`.
</span>

In [1]:
import sqlite3

# cоздаём соединение с базой данных
conn = sqlite3.connect('data/checking-logs.sqlite')

<span style="color: green; font-size: 13px;">

2. Получи схему таблицы `test`.
</span>

In [2]:
# получаем схему таблицы test
query_schema = "PRAGMA table_info(test);"
schema = conn.execute(query_schema).fetchall()
schema

[(0, 'uid', 'TEXT', 0, None, 0),
 (1, 'labname', 'TEXT', 0, None, 0),
 (2, 'first_commit_ts', 'TIMESTAMP', 0, None, 0),
 (3, 'first_view_ts', 'TIMESTAMP', 0, None, 0)]

<span style="color: green; font-size: 13px;">

3. Получи только первые `10` строк таблицы `test`, чтобы проверить, как она выглядит.
</span>

In [3]:
# получаем первые 10 строк таблицы test
query_data = "SELECT * FROM test LIMIT 10;"
data = conn.execute(query_data).fetchall()
for row in data:
    print(row)

('user_1', 'laba04', '2020-04-26 17:06:18', '2020-04-26 21:53:59')
('user_1', 'laba04s', '2020-04-26 17:12:11', '2020-04-26 21:53:59')
('user_1', 'laba05', '2020-05-02 19:15:18', '2020-04-26 21:53:59')
('user_1', 'laba06', '2020-05-17 16:26:35', '2020-04-26 21:53:59')
('user_1', 'laba06s', '2020-05-20 12:23:37', '2020-04-26 21:53:59')
('user_1', 'project1', '2020-05-14 20:56:08', '2020-04-26 21:53:59')
('user_10', 'laba04', '2020-04-25 08:24:52', '2020-04-18 12:19:50')
('user_10', 'laba04s', '2020-04-25 08:37:54', '2020-04-18 12:19:50')
('user_10', 'laba05', '2020-05-01 19:27:26', '2020-04-18 12:19:50')
('user_10', 'laba06', '2020-05-19 11:39:28', '2020-04-18 12:19:50')


<span style="color: green; font-size: 13px;">
    
4. Найди среди всех пользователей минимальное значение этой самой дельты (периода времени от даты первого коммита
пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос.
    - Для этого сджойни данные своей таблицы с данными таблицы `deadlines`.
    - Период времени должен отображаться в часах.
    - Не учитывай лабораторное задание `project1` с более длительным сроком сдачи, поскольку оно будет выделяться на фоне других.
    - Сохрани значение в датафрейме `df_min` с соответствующим `uid`.
</span>

In [4]:
import pandas as pd

# находим минимальное значение периода времени для каждого пользователя
query = """
SELECT
    test.uid,
    MIN((strftime('%s', test.first_commit_ts) - deadlines.deadlines) / 3600) AS min_time_diff
FROM
    (SELECT * FROM test WHERE labname NOT LIKE 'project1') AS test
LEFT JOIN
    deadlines ON test.labname = deadlines.labs;
"""
# результат запроса в датафрейм
df_min = pd.io.sql.read_sql(query, conn)
df_min

,uid,min_time_diff
0,user_30,-202


<span style="color: green; font-size: 13px;">
    
5. Выполни те же самые операции для максимального значения дельты, используя только **один** запрос. Название итогового датафрейма — `df_max`.
</span>

In [5]:
# находим максимальное значение периода времени для каждого пользователя
query = """
SELECT
    test.uid,
    MAX((strftime('%s', test.first_commit_ts) - deadlines.deadlines) / 3600) AS max_time_diff
FROM
    (SELECT * FROM test WHERE labname NOT LIKE 'project1') AS test
LEFT JOIN
    deadlines ON test.labname = deadlines.labs;
"""
# результат запроса в датафрейм
df_max = pd.io.sql.read_sql(query, conn)
df_max

,uid,max_time_diff
0,user_25,-2


<span style="color: green; font-size: 13px;">
    
6. Выполни те же самые операции для среднего значения дельты, используя только один запрос. На этот раз ваш итоговый датафрейм не должен включать столбец `uid`; имя датафрейма — `df_avg`.
</span>

In [6]:
# рассчитываем среднее значение дельты для каждого пользователя
query = """
SELECT
    AVG((strftime('%s', test.first_commit_ts) - deadlines.deadlines) / 3600) AS avg_delta
FROM
    (SELECT * FROM test WHERE labname NOT LIKE 'project1') AS test
LEFT JOIN
    deadlines ON test.labname = deadlines.labs;
"""
# результат запроса в датафрейм
df_avg = pd.io.sql.read_sql(query, conn)
df_avg

,avg_delta
0,-89.125


<span style="color: green; font-size: 13px;">
    
7. Мы хотим проверить предположение о том, что у пользователей, посетивших ленту новостей всего несколько раз, период времени между датой первого коммита и сроком сдачи лабораторного задания меньше. Для этого тебе необходимо рассчитать коэффициент корреляции между количеством просмотров страниц и разницей между датами.
    - Используя только **один** запрос, создай таблицу со столбцами: `uid`, `avg_diff`, `pageviews`.
        - `uid` — это uid, существующие в таблице `test`.
        - `avg_diff` — среднее значение дельты (периода времени между датой первого коммита и сроком сдачи лабораторного задания) для каждого пользователя.
        - `pageviews` — количество посещений ленты новостей одним пользователем.
    - Не учитывай лабораторное задание `project1`.
    - Сохрани результаты в датафрейме `views_diff`.
    - Используй метод `corr()` библиотеки Pandas для вычисления коэффициента корреляции между количеством просмотров и значением дельты.
</span>

In [7]:
query = """
SELECT
    t.uid, t.avg_diff, p.pageviews
FROM (
    SELECT test.uid, AVG((strftime('%s', test.first_commit_ts) - deadlines.deadlines) / 3600) AS avg_diff
    FROM test
    LEFT JOIN deadlines ON test.labname = deadlines.labs
    WHERE labname NOT LIKE 'project1'
    GROUP BY test.uid
) t
LEFT JOIN (
    SELECT uid, COUNT(*) AS pageviews
    FROM pageviews
    WHERE uid LIKE 'user_%'
    GROUP BY uid
) p ON t.uid = p.uid;
"""

# результат запроса в датафрейм
views_diff = pd.io.sql.read_sql(query, conn)

print("Коэффициент корреляции:")
print(views_diff.corr(numeric_only=True))

Коэффициент корреляции:
           avg_diff  pageviews
avg_diff   1.000000  -0.279736
pageviews -0.279736   1.000000


<span style="color: green; font-size: 13px;">

4. Закрой соединение.
</span>
<span style="color: green; font-size: 12px;">

Небольшой совет — выполняй все операции поочередно, от простой к более сложной, а не пытаясь сделать всё вместе и сразу. Это поможет в отладке твоих запросов.
</span>

In [8]:
# закрываем соединение
conn.close()